# TensorFlow: DNN using TF dataset API

Demonstrates creating DNN to accomplish image classification with Conv2D and MaxPool2D layers.
Model is trained on horses-and-humans dataset.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from common import CV_DATA_DIR

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf
import tensorflow_datasets as tfds

print("TF Version: ", tf.__version__)
print("TF Eager mode: ", tf.executing_eagerly())
print("TF GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")

TF Version: 2.20.0
TF Devices: ['CPU', 'GPU']


# Prepare dataset

In [ ]:
# Load datase
(train_ds, test_ds), ds_info = tfds.load(
    'horses_or_humans',
    split=['train', 'test'],
    batch_size=32,
    with_info=True,
    as_supervised=True)

In [ ]:
print(f'Number of training examples: {ds_info.splits['train'].num_examples}')
print(f'Number of testing examples: {ds_info.splits['test'].num_examples}')
print(f'Number of classes: {ds_info.features['label'].num_classes}')
print(f'Image shape: {ds_info.features['image'].shape}')

In [ ]:
VAL_DIR = CV_DATA_DIR / 'animals' / 'horses-and-humans' / 'val'
BUFFER_SIZE = 1000

# Rescale train dataset
train_ds = (train_ds
    .cache()
    .shuffle(BUFFER_SIZE)
    .prefetch(tf.data.AUTOTUNE))

# Rescale test dataset
test_ds = (test_ds
   .cache()
   .prefetch(BUFFER_SIZE))

valid_ds = None
if VAL_DIR.is_dir():
    valid_ds = tf.keras.utils.image_dataset_from_directory(
        VAL_DIR,
        image_size=(300, 300),
        batch_size=32,
        label_mode='binary')
    valid_ds = (valid_ds
        .cache()
        .prefetch(BUFFER_SIZE))

In [ ]:
# Take one batch from dataset
sample_batch = list(train_ds.take(1))[0]

image_batch = sample_batch[0]
label_batch = sample_batch[1]

# Extract images and labels from batch
print(f'Image batch shape: {image_batch.shape}')
print(f'Label batch shape: {label_batch.shape}')
print(f'Max value: {np.max(image_batch[0].numpy())}')
print(f'Min value: {np.min(image_batch[0].numpy())}')

# Define model

In [ ]:
# Define model
model = tf.keras.models.Sequential([
    # Input with 300x300x3 shape
    tf.keras.Input(shape=(300, 300, 3)),
    # Rescale input volume
    tf.keras.layers.Rescaling(scale=1./255),
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Binary classification: 0 - 'horses', 1 - 'humans'
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Output model summary
model.summary()

In [ ]:
# Compile model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

# Train model

In [ ]:
class StopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Print available metric
        keys = list(logs.keys())
        print('End epoch {} of training; got log keys: {}'.format(epoch, keys))
        # Check if we reached desired accuracy value
        if logs.get('accuracy') >= 0.99:
            print('\nReached 90% accuracy so cancelling training!')
            self.model.stop_training = True

%%time
history = model.fit(
    train_ds,
    epochs=15,
    callbacks=[StopCallback()],
    verbose=2,
    validation_data=valid_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
if valid_ds is not None:
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.show()

In [ ]:
# Evaluate on test set
model.evaluate(test_ds, verbose=2)